In [6]:
import os
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()
documents = LlamaParse(result_type="markdown")

In [8]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

documents = LlamaParse(result_type="markdown")


def pdf_parser(pdf_file_path: str):
    """
    PDF 파일을 파싱하여 그 내용을 Markdown 파일로 저장합니다.

    Args:
        pdf_file_path (str): 처리할 PDF 파일의 경로.
    """
    print(f"🔄 '{pdf_file_path}' 파일 파싱을 시작합니다...")

    try:
        # parsing instruction 을 지정합니다.
        parsing_instruction = (
            "You are parsing a AI Report. Please extract tables in markdown format."
        )

        # LlamaParse 설정
        parser = LlamaParse(
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
            result_type="markdown",
            # parsing_mode="Unstructured",
            language="ko",
            parsing_instruction=parsing_instruction,
        )

        # 1. LlamaParse를 사용하여 PDF 파일을 로드합니다.
        # 'documents' 객체는 이 함수 외부에서 미리 정의되어 있어야 합니다.
        parsed_docs = documents.load_data(file_path=pdf_file_path)

        # 2. LangChain 형식의 도큐먼트로 변환합니다.
        docs = [doc.to_langchain_format() for doc in parsed_docs]
    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다 - {pdf_file_path}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
    return docs

In [14]:
# --- 함수 사용 예시 ---
# 이 코드를 실행하기 전에 'documents' 파서 객체를 초기화해야 합니다.
# file_to_parse = "data/디지털정부혁신추진계획.pdf"
file_to_parse = "./data/appendix-keywords-edit.pdf"
docs = pdf_parser(file_to_parse)

🔄 './data/appendix-keywords-edit.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id e65fd331-b3f3-449b-b33d-777ac4eb4545


In [15]:
docs[0].page_content

'# Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\n# Embedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\n# Token\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\n# Tokenizer\n\n정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\n# VectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.\n\n예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.\n\n연관키워드: 임베딩, 데이터베이스, 벡터화\n\n# SQL\n\n정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회,

In [16]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 문장을 나누는 구분자 설정
    chunk_size=250,  # 각 청크의 최대 길이 설정
    chunk_overlap=0,  # 청크 간의 겹치는 부분 설정
    length_function=len,  # 길이를 계산하는 함수 설정
)

In [17]:
myDocs = text_splitter.split_documents(docs)

In [19]:
print(myDocs[0])

page_content='# Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.

예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.

연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

# Embedding'
